# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
## start session 

import findspark
findspark.init('/home/ian/spark-4.0.0-bin-hadoop3')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Hyundai_project').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/14 11:41:08 WARN Utils: Your hostname, ian-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/08/14 11:41:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/14 11:41:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/14 11:41:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
## read data in 

df = spark.read.csv('cruise_ship_info.csv', inferSchema=True, header=True)

## check data
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [4]:
## check some examples 
df.head(3)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7)]

## handling categorical variable: cruise line

In [9]:
cruise_line_counts = df.groupby('Cruise_line').count()
cruise_line_counts.sort(cruise_line_counts['count'].desc()).show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|  Royal_Caribbean|   23|
|         Carnival|   22|
|         Princess|   17|
| Holland_American|   14|
|        Norwegian|   13|
|            Costa|   11|
|        Celebrity|   10|
|              MSC|    8|
|              P&O|    6|
|             Star|    6|
|Regent_Seven_Seas|    5|
|        Silversea|    4|
|           Cunard|    3|
|         Seabourn|    3|
|         Windstar|    3|
|          Oceania|    3|
|          Crystal|    2|
|           Disney|    2|
|          Azamara|    2|
|           Orient|    1|
+-----------------+-----+



In [10]:
from pyspark.sql.functions import count_distinct

In [12]:
## how many distinct lines are there
df.select(count_distinct('Cruise_line')).show()


+---------------------------+
|count(DISTINCT Cruise_line)|
+---------------------------+
|                         20|
+---------------------------+



In [13]:
## handling categorical 

from pyspark.ml.feature import StringIndexer

In [16]:
cl_indexer = StringIndexer(inputCol = 'Cruise_line', outputCol= 'Cruise_line_index')
df_indexed = cl_indexer.fit(df).transform(df)


In [17]:
## check results 
df_indexed.head(3)


[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_index=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, Cruise_line_index=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, Cruise_line_index=1.0)]

## data preprocessing

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler ## for our features

In [26]:
df_indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'Cruise_line_index']

In [33]:
## init the object
assembler = VectorAssembler(inputCols=
                            ['Age',
                             'Tonnage',
                             'passengers',
                             'length',
                             'cabins',
                             'passenger_density',
                             # 'crew',
                             'Cruise_line_index'], 
                            outputCol= 'features') ## we name the combined col 'features'

In [34]:
preprocessed = assembler.transform(df_indexed)

In [36]:
preprocessed.select('features', 'crew').show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows


In [37]:
final_data = preprocessed.select('features', 'crew')

In [38]:
train, test = final_data.randomSplit([0.8, 0.2])

In [41]:
# check 
train.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               120|
|   mean|  8.00100000000001|
| stddev|3.6072565519980384|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



## lin reg

In [42]:
from pyspark.ml.regression import LinearRegression

In [43]:
lr = LinearRegression(labelCol='crew')

In [44]:
model = lr.fit(train)

25/08/14 11:57:01 WARN Instrumentation: [f76bbbc1] regParam is zero, which might cause numerical instability and overfitting.
25/08/14 11:57:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/08/14 11:57:02 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [45]:
## see some results 
model.coefficients

DenseVector([-0.0166, 0.0022, -0.1429, 0.4236, 0.895, -0.001, 0.0455])

In [48]:
## get evaluation on test 
test_results = model.evaluate(test)

In [52]:
print('RMSE')
print(f'test: {test_results.rootMeanSquaredError}')
print(f'train: {model.summary.rootMeanSquaredError}')


RMSE
test: 0.7351394403262481
train: 1.0042285899144316


In [54]:
print('adj R^2')
print(f'test: {test_results.r2adj}')
print(f'train: {model.summary.r2adj}')


adj R^2
test: 0.9291078314036639
train: 0.9169624887474307


### checking some corr

In [55]:
from pyspark.sql.functions import corr

In [57]:
df.select(corr('crew', 'passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [59]:
for col in ['Age','Tonnage','passengers','length','cabins','passenger_density',
                             'Cruise_line_index']:
    print(f'corr btn crew and {col}')
    print(f'{df.select(corr('crew', col)).collect()[0][0]:.2f} <n')

corr btn crew and Age
-0.5306565039638852
corr btn crew and Tonnage
0.927568811544939
corr btn crew and passengers
0.9152341306065384
corr btn crew and length
0.8958566271016579
corr btn crew and cabins
0.9508226063578497
corr btn crew and passenger_density


{"ts": "2025-08-14 12:01:52.593", "level": "ERROR", "logger": "DataFrameQueryContextLogger", "msg": "[UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Cruise_line_index` cannot be resolved. Did you mean one of the following? [`Cruise_line`, `Ship_name`, `passengers`, `Tonnage`, `cabins`]. SQLSTATE: 42703", "context": {"file": "jdk.internal.reflect.GeneratedMethodAccessor94.invoke(Unknown Source)", "line": "", "fragment": "col", "errorClass": "UNRESOLVED_COLUMN.WITH_SUGGESTION"}, "exception": {"class": "Py4JJavaError", "msg": "An error occurred while calling o36.select.\n: org.apache.spark.sql.AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Cruise_line_index` cannot be resolved. Did you mean one of the following? [`Cruise_line`, `Ship_name`, `passengers`, `Tonnage`, `cabins`]. SQLSTATE: 42703;\n'Project [unresolvedalias('corr(crew#25, 'Cruise_line_index))]\n+- Relation [Ship_name#17,Cruise

-0.15550928421699717
corr btn crew and Cruise_line_index


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column, variable, or function parameter with name `Cruise_line_index` cannot be resolved. Did you mean one of the following? [`Cruise_line`, `Ship_name`, `passengers`, `Tonnage`, `cabins`]. SQLSTATE: 42703;
'Project [unresolvedalias('corr(crew#25, 'Cruise_line_index))]
+- Relation [Ship_name#17,Cruise_line#18,Age#19,Tonnage#20,passengers#21,length#22,cabins#23,passenger_density#24,crew#25] csv
